In [56]:
import pandas as pd 
import numpy as np
import pytest
import ipytest 
ipytest.autoconfig()

In [57]:
# created data
data = pd.read_csv('predicted_ratings.csv')
data.head(3)

,Unnamed: 0,value,value.1,value.2,value.3,value.4,value.5,value.6,value.7,value.8,...,value.24,value.25,value.26,value.27,value.28,value.29,value.30,value.31,value.32,value.33
0,movie_id,1.000000,2.000000,3.000000,4.000000,5.000000,6.00000,7.00000,8.00000,9.000000,...,25.000000,26.000000,27.000000,28.000000,29.000000,30.000000,31.000000,32.000000,33.000000,34.000000
1,user_id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,4.003192,3.920304,4.155777,3.527676,3.920494,4.14339,2.65563,4.06705,4.010433,...,3.797145,3.984571,3.331073,3.990534,3.457581,3.104856,2.720677,3.352794,3.042513,4.565993


In [58]:
def find_outliers(data):
    """Find Outliers in data, retur indices of outiliers"""
    out = data[(data - data.mean()).abs() > 2 * data.std()]
    return out.index

In [59]:
find_outliers(data=data)

/var/folders/5v/6sdcd_ln4fxb_wj2944sknw00000gn/T/ipykernel_76217/314043693.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  out = data[(data - data.mean()).abs() > 2 * data.std()]
/var/folders/5v/6sdcd_ln4fxb_wj2944sknw00000gn/T/ipykernel_76217/314043693.py:3: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  out = data[(data - data.mean()).abs() > 2 * data.std()]
/var/folders/5v/6sdcd_ln4fxb_wj2944sknw00000gn/T/ipykernel_76217/314043693.py:3: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and w

RangeIndex(start=0, stop=102, step=1)

In [60]:
# created a new data.
data2 = pd.Series(np.random.randint(50, 60, 10_000))

In [61]:
# creating outliers
data2[7] = 3
data2[1003] = 100

In [62]:
# Using the function and check the time
%timeit find_outliers(data)

/var/folders/5v/6sdcd_ln4fxb_wj2944sknw00000gn/T/ipykernel_76217/314043693.py:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  out = data[(data - data.mean()).abs() > 2 * data.std()]
/var/folders/5v/6sdcd_ln4fxb_wj2944sknw00000gn/T/ipykernel_76217/314043693.py:3: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  out = data[(data - data.mean()).abs() > 2 * data.std()]
/var/folders/5v/6sdcd_ln4fxb_wj2944sknw00000gn/T/ipykernel_76217/314043693.py:3: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and w

2.17 ms ± 53.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
#check the time
%timeit find_outliers(data2)

279 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [64]:
# generate data and the outliers
def gen_data(size, num_outliers):
    """Generate data in with size element containint num_outliers outliers.
    Return the data and the outliers"""
    regular  = np.random.randint(50, 60, size-num_outliers)
    low = np.random.randint(1, 10, num_outliers//2)
    high = np.random.randint(100, 110, num_outliers-len(low))

    data = np.concatenate([regular,low, high])
    np.random.shuffle(data)
    return pd.Series(data), pd.Series(np.concatenate([low,high]))

In [65]:
gen_data(2, 2)

(0    103
 1      7
 dtype: int64,
 0      7
 1    103
 dtype: int64)

In [66]:
def test_bench_outliers(benchmark):
    size = 10_000 # Usual size of data
    num_outliers = 5 # Usual number of outliers
    data, expected = gen_data(size=size, num_outliers=num_outliers)
    indices = benchmark(find_outliers, data2)
    outliers = data.loc[indices]
    assert set(expected) == set(outliers), 'bad results'